In [ ]:
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!python -m pip install --upgrade pip
!pip -V
!pip install scikit-build
!git clone https://github.com/opencv/opencv-python
!cd opencv-python && ENABLE_CONTRIB=1 python setup.py bdist_wheel -- -DWITH_CUDA=ON -DENABLE_FAST_MATH=1 -DCUDA_FAST_MATH=1 -DWITH_CUBLAS=1 -DCUDA_ARCH_BIN='6.1 7.5' -- -j8

In [ ]:
# import os
# dir_name = 'opencv-python/dist'
# for dir in os.listdir(dir_name):
#     file = os.path.join(dir_name, dir)
#     print(file)

!pip install opencv-python/dist/opencv_contrib_python-4.5.4.60-cp37-cp37m-linux_x86_64.whl

In [ ]:
import cv2
cv2.__version__

!nvidia-smi

In [ ]:
!sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!sudo chmod a+rx /usr/local/bin/youtube-dl
!youtube-dl -f '(bestvideo+bestaudio/best)'  -ciw -o "%(02-10-2021)s.%(mp4)s" -v 'https://www.youtube.com/watch?v=Ssr-OCU5EGk'
%mv 'NA.NA.mp4' 'video.mp4'

In [ ]:
config_file = '/content/drive/My Drive/backup/yolo-obj.cfg'
weight_file = '/content/drive/My Drive/backup/yolo-obj_best.weights'
class_names = '/content/drive/My Drive/backup/obj.names'
video_file = 'video.mp4'

!sed -i 's/width=608/width=1280/g' '/content/drive/My Drive/backup/yolo-obj.cfg'
!sed -i 's/height=608/height=736/g' '/content/drive/My Drive/backup/yolo-obj.cfg'

In [ ]:
import time
Conf_threshold = 0.25
NMS_threshold = 0.4
COLORS = [(0, 255, 0), (0, 0, 255), (255, 0, 0),
          (255, 255, 0), (255, 0, 255), (0, 255, 255)]

class_name = []
with open(class_names, 'r') as f:
    class_name = [cname.strip() for cname in f.readlines()]
# print(class_name)
net = cv2.dnn.readNet(weight_file, config_file)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(1280, 736), scale=1/255, swapRB=True)

In [ ]:
cap = cv2.VideoCapture(video_file)
starting_time = time.time()
frame_counter = 0
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_counter += 1
    if ret == False:
        break
    classes, scores, boxes = model.detect(gray, Conf_threshold, NMS_threshold)
    for (classid, score, box) in zip(classes, scores, boxes):
        color = COLORS[int(classid) % len(COLORS)]
        label = "%s : %f" % (class_name[classid[0]], score)
        cv2.rectangle(gray, box, color, 1)
        cv2.putText(gray, label, (box[0], box[1]-10),
                   cv2.FONT_HERSHEY_COMPLEX, 0.3, color, 1)
    endingTime = time.time() - starting_time
    fps = frame_counter/endingTime
    print(f"FPS = {fps}")
    cv2.putText(gray, f'FPS: {fps}', (20, 50),
               cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
    #cv2.imshow('frame', gray)
    #key = cv2.waitKey(1)
    key = 0
    if key == ord('q'):
        break
cap.release()
#cv2.destroyAllWindows()